<a href="https://colab.research.google.com/github/AadityaPanchakshikar/Technical-Analysis-w-py/blob/main/Supertrend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supertrend

-  Supertrend is a trend-following indicator<br>
-  Uses Average True Range (ATR)
-  Plotted either above or below the closing price
-  Used primarily for Intra-day
-  Uses Parameters ATR Period (7) and multiplier (3)
-  Perfect for trending Markets

### Formula
##### Upper Band = (High + Low)/ 2 + multiplier * ATR
##### Lower Band = (High + Low)/ 2 - multiplier * ATR
<br>
<br>

In [ ]:
import pandas as pd
import numpy as np

# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

import plotly.graph_objects as go
file_path = '/content/drive/MyDrive/Colab Notebooks/Trading/Data/SampleNiftyData.csv'
df = pd.read_csv(file_path, parse_dates=["date"], index_col="date")
df.head(2)

In [ ]:
def ATR(df, range_period):
   df['High-Low']=df['high']-df['low']
    df['High-Previous_Close']=abs(df['high']-df['close'].shift(1))
    df['Low-Previous_Close']=abs(df['low']-df['close'].shift(1))
    df['True_Range']=df[['High-Low','High-Previous_Close','Low-Previous_Close']].max(axis=1,skipna=False)
    df['ATR'] = df['True_Range'].ewm(com=range_period,min_periods=range_period).mean()
    return df

In [ ]:
range_period = 7
multiplier = 3
df = ATR(df, range_period)
df.dropna(inplace=True)
df[['close', 'ATR']].tail(2)

In [ ]:
df['basic_upper_band'] = (df.high + df.low) / 2 + multiplier * df['ATR']

In [ ]:
df['final_upper_band'] = 0.00

for i in range(range_period,len(df)):

    if df['close'][i-1]<=df['final_upper_band'][i-1]:
        # Shift band downwards if basic_upper_band is lower
        df['final_upper_band'].iat[i] = min(df['basic_upper_band'].iat[i],df['final_upper_band'].iat[i-1])
    else:
        # Keep Basic Band Value
        df['final_upper_band'].iat[i] = df['basic_upper_band'].iat[i]

df.loc[df.index >= '2022-07-06',['close', 'final_upper_band', 'basic_upper_band']]

In [ ]:
df['basic_lower_band'] = (df.high + df.low) / 2 - multiplier * df['ATR']
df['final_lower_band'] = 0.00

In [ ]:
for i in range(range_period,len(df)):

    if df['close'][i-1]>=df['final_lower_band'][i-1]:
        df['final_lower_band'].iat[i] = max(df['basic_lower_band'].iat[i],df['final_lower_band'].iat[i-1])
                                        # Shift band upward if basic_lower_band is higher
    else:
        df['final_lower_band'].iat[i] = df['basic_lower_band'].iat[i] #keep Same

df[['close', 'final_lower_band', 'basic_lower_band']].tail(10)

<hr>
<b><i> Calculate Super Trend for first row</i></b><br>
-  if Previous close <= Previous FINAL UPPERBAND:<br>
&emsp;&emsp;&emsp;&emsp;SuperTrend = final_upper_band<br>
-  else:<br>
&emsp;&emsp;&emsp;&emsp;SuperTrend = final_lower_band

In [ ]:
df['SuperTrend']=np.nan
if df['close'].iat[range_period]<=df['final_upper_band'].iat[range_period]:
    df['SuperTrend'].iat[range_period+1]=df['final_upper_band'].iat[range_period+1]
else:
    df['SuperTrend'].iat[range_period+1]=df['final_lower_band'].iat[range_period+1]

df[['close','final_upper_band', 'final_lower_band', 'SuperTrend']].head(10)

<hr>
<b><i>Calculate Super Trend for rest of the rows</b></i>

In [ ]:
for i in range(range_period,len(df)):
    if df['SuperTrend'].iat[i-1]==df['final_upper_band'].iat[i-1] and df['close'].iat[i]<=df['final_upper_band'].iat[i]:
        df['SuperTrend'].iat[i]=df['final_upper_band'].iat[i] # Same Trend
    elif  df['SuperTrend'].iat[i-1]==df['final_upper_band'].iat[i-1] and df['close'][i]>=df['final_upper_band'].iat[i]:
        df['SuperTrend'].iat[i]=df['final_lower_band'].iat[i] # Trend Changed
    elif df['SuperTrend'].iat[i-1]==df['final_lower_band'].iat[i-1] and df['close'][i]>=df['final_lower_band'].iat[i]:
        df['SuperTrend'].iat[i]=df['final_lower_band'].iat[i] # Same Trend
    elif df['SuperTrend'].iat[i-1]==df['final_lower_band'][i-1] and df['close'][i]<=df['final_lower_band'].iat[i]:
        df['SuperTrend'].iat[i]=df['final_upper_band'].iat[i] # Trend Changed
df.dropna(inplace=True)

In [ ]:
df[['close', 'SuperTrend']].tail(10)

## <b><i>Display in a chart</i></b>

In [ ]:
df['direction'] = np.where(df.close >= df.SuperTrend, 'bullish','bearish')
df['up'] = np.where(df.direction == 'bullish', df.SuperTrend, np.nan)
df['down'] = np.where(df.direction == 'bearish', df.SuperTrend, np.nan)
df['change'] = np.where((df.direction == 'bullish') & (df.direction.shift(1) == 'bearish'), 1, 0)
df['change'] = np.where((df.direction == 'bearish') & (df.direction.shift(1) == 'bullish'), -1, df.change)
df['change'] = df['change'].astype(int)
df[['close', 'SuperTrend', 'direction', 'up', 'down', 'change']].sample(10)

In [ ]:
df['close2'] = np.where(df.change == 0, 0, df.close)
df['close2'] = np.where(df.change == -1, df.final_upper_band, df.close2)
df['close2'] = np.where(df.change == 1, df.final_lower_band, df.close2)
df['close3'] = df['close2']
df['close3'] = np.where(df.close3 != 0, df.close3 + 150, df.close3)
df['close4'] = df['close2']
df['close4'] = np.where(df.close4 != 0, df.close4 - 150, df.close4)